In [ ]:
import pathlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
def accumulate_net_data_frames(so_far, element):
    return toolz.assoc(so_far,  element.ObjectId, element)

def id_net_data_frame_map(project):
    return toolz.pipe(project.DataFrames.Items,
                   list,
                   lambda dfs: toolz.reduce(accumulate_net_data_frames, dfs, {})
              )

net_data_frames = toolz.valmap(id_net_data_frame_map, projects)
net_data_frames

In [ ]:
def identify_net_data_frame(item):
    df_id, data_frame = item
    return {'id': data_frame.ObjectId, 'name': data_frame.Name, 'display_name': data_frame.DisplayName}

def identify_net_data_frames(data_frames):
    return toolz.pipe(data_frames,
                      # toolz.do(lambda df: print(df)),
                      lambda dfs: toolz.map(identify_net_data_frame, dfs.items()),
                      list)


identifications = toolz.valmap(identify_net_data_frames, net_data_frames)
identifications

In [ ]:
@toolz.curry
def add_project_name(pn, df_ids):
    return toolz.assoc(df_ids, 'project', pn)
    
    
def summarize_net_data_frames(item):
    pn, dfs_ids = item
    result = toolz.pipe(dfs_ids,
                        # toolz.do(lambda df_id: print(df_id)),
                        toolz.map(lambda df_ids: add_project_name(pn, df_ids)),
                        list,
                       )
    return pn, result

id_summaries = toolz.itemmap(summarize_net_data_frames, identifications)
id_summaries

In [ ]:
col_names = ['project', 'id', 'name', 'display_name']

In [ ]:
pd.DataFrame(data=id_summaries['permian-a'], columns=col_names)

In [ ]:
pd.DataFrame(data=id_summaries['gng'], columns=col_names)

In [ ]:
@toolz.curry
def find_net_data_frame_by_name(project_name, net_dfs, name):
    project_net_dfs = toolz.get(project_name, net_dfs)
    candidates = toolz.pipe(project_net_dfs,
                            toolz.valfilter(lambda df: df.Name == name),
                            lambda cs: cs.values(),
                            list,
                           )
    assert len(candidates) == 1
    return candidates[0]

@toolz.curry
def find_net_data_frames(project_name, data_frame_names):
    return toolz.pipe(data_frame_names,
                      toolz.map(find_net_data_frame_by_name(project_name, net_data_frames)),
                      list,
                      )

In [ ]:
find_permian_data_frames = find_net_data_frame_by_name('permian-a', net_data_frames)
find_permian_data_frames('C3-Microseismic Data Frame 01')

In [ ]:
# Adapted from code at
# https://docs.microsoft.com/en-us/dotnet/framework/data/adonet/dataset-datatable-dataview/creating-a-datareader
# retrieved on 18-Apr-2021.

def row_as_dict(reader):
    return {reader.GetName(i): reader[reader.GetName(i)] for i in range(reader.FieldCount)}

def read_data_table(data_table):
    reader = data_table.CreateDataReader()
    try:
        while True:
            if reader.HasRows:
                has_row = reader.Read()
                while has_row:
                    yield row_as_dict(reader)
                    has_row = reader.Read()
            else:
                return
            if not reader.NextResult():
                break
    finally:
        reader.Dispose()

In [ ]:
net_data_frame_names = {
    'permian-a': [
        'Project Data Frame 01',
        'FDI Observations',
        'C3-Microseismic Data Frame 01',
    ],
    'gng': [
        'Project Data Frame 01',
        'Fault Trace Set Data Frame 01',
        'Stage Data Frame 01',
        'Well Log Set Data Frame 01',
        'Horizon Marker Set Data Frame01',
    ],
}

In [ ]:
def table_data_frame(data_table):
    return pd.DataFrame(data=[r for r in read_data_table(data_table)])

def obs_data_frames_for_project(project_name):
    result = toolz.pipe(
        net_data_frame_names,
        toolz.get(project_name),
        find_net_data_frames(project_name),
        toolz.map(lambda df: df.DataTable),
        toolz.map(table_data_frame),
        list,
    )
    return result

In [ ]:
obs_data_frames = {pn: obs_data_frames_for_project(pn) for pn in projects.keys()}

In [ ]:
def data_frames_for_project(item):
    project_name, net_data_frames_map = item
    mapped_data_frames = toolz.pipe(
        net_data_frames_map,
        toolz.valmap(toolz.compose(table_data_frame, lambda df: df.DataTable)),
    )
    return project_name, mapped_data_frames

data_frames = toolz.pipe(
    net_data_frames,
    toolz.itemmap(data_frames_for_project),
    # list,
)
data_frames

In [ ]:
permian_data_frames = obs_data_frames['permian-a']
len(permian_data_frames)

In [ ]:
permian_data_frames[0]

In [ ]:
permian_data_frames[1]

In [ ]:
permian_data_frames[2]

In [ ]:
gng_data_frames = obs_data_frames['gng']
len(gng_data_frames)

In [ ]:
gng_data_frames[0]

In [ ]:
gng_data_frames[1]

In [ ]:
gng_data_frames[2]

In [ ]:
gng_data_frames[3]

In [ ]:
gng_data_frames[4]